<a href="https://colab.research.google.com/github/wilson1yan/VideoGPT/blob/master/notebooks/Using_VideoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using VideoGPT
This is a notebook demonstrating how to use VideoGPT and any pretrained models, Make sure that it is a GPU instance: **Change Runtime Type -> GPU**

## Installation
First, we install the necessary packages

In [3]:
#comment out to use the source code on local machine
# ! pip install git+https://github.com/wilson1yan/VideoGPT.git
! pip install scikit-video av
! pip install --upgrade --no-cache-dir gdown
! pip install -r requirements.txt
! pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 69.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 75.7 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.6 MB/s eta 0:00:00
  Using cached pytorch_lightning-2.0.1-py3-none-any.whl (716 kB)
  Using cached gradio-3.24.1-py3-none-any.whl (15.7 MB)
  Using cached moviepy-1.0.3.tar.gz (388 kB)
  Preparing metadata (setup.py) ... done
  Using cached imageio-2.27.0-py3-none-any.whl (3.4 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 181.2 MB/s eta 0:00:00
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
  Using cached nvidia_cusparse_cu11-11.7.4.91-py3-none-manylinux1_x86_64.whl (173.2 MB)
  Using cached nvidia_cuda_cupti_cu11-11.7.1

In [4]:
%matplotlib inline

from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import torchmetrics
print(torchmetrics.__version__)

import os
import torch
from torchvision.io import read_video, read_video_timestamps


from videogpt import download, load_vqvae, load_videogpt
from videogpt import download, load_videogpt
from videogpt.data import preprocess

VIDEOS = {
    'breakdancing': '1OZBnG235-J9LgB_qHv-waHZ4tjofiDgj',
    'bear': '16nIaqq2vbPh-WMo_7hs9feVSe0jWVXLF',
    'jaywalking': '1UxKCVrbyXhvMz_H7dI4w5hjPpRGCAApy',
    'cartoon': '1ONcTMSEuGuLYIDbX-KeFqd390vbTIH9d'
}

ROOT = 'pretrained_models'

0.11.4


## Downloading a Pretrained VQ-VAE
There are four pretrained models available: `bair_stride4x2x2`, `ucf101_stride4x4x4`, `kinetics_stride4x4x4`, and `kinetics_stride2x4x4`. BAIR was trained on 64 x 64 video, and the rest on 128 x 128. The `stride` component represents the THW downsampling the VQ-VAE performs on the video tensor.

In [5]:
device = torch.device('cuda')
vqvae = load_vqvae('kinetics_stride2x4x4', device=device, root=ROOT).to(device)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1jvtjjtrtE4cy6pl7DK_zWFEPY3RZt2pB
From (redirected): https://drive.google.com/uc?id=1jvtjjtrtE4cy6pl7DK_zWFEPY3RZt2pB&confirm=t&uuid=a662f50c-203b-404e-b33c-1add0e54935c
To: /home/ubuntu/MIRAI/VideoGPT/pretrained_models/kinetics_stride2x4x4
100%|██████████| 258M/258M [00:04<00:00, 53.7MB/s] 
/home/ubuntu/miniconda3/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.1.6 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.u

## Video Loading and Preprocessing
The code below downloads, loads, and preprocesses a given `mp4` file.

In [6]:
video_name = 'jaywalking'
# `resolution` must be divisible by the encoder image stride
# `sequence_length` must be divisible by the encoder temporal stride
resolution, sequence_length = vqvae.args.resolution, 16

video_filename = download(VIDEOS[video_name], f'{video_name}.mp4')
pts = read_video_timestamps(video_filename, pts_unit='sec')[0]
video = read_video(video_filename, pts_unit='sec', start_pts=pts[0], end_pts=pts[sequence_length - 1])[0]
video = preprocess(video, resolution, sequence_length).unsqueeze(0).to(device)

Downloading...
From: https://drive.google.com/uc?id=1UxKCVrbyXhvMz_H7dI4w5hjPpRGCAApy
To: /home/ubuntu/.cache/videogpt/jaywalking.mp4
100%|██████████| 3.29M/3.29M [00:00<00:00, 24.8MB/s]


## VQ-VAE Encoding and Decoding
Now, we can encode the video through the `encode` function. The `encode` function also has an optional input `including_embeddings` (default `False`) which will also return the embedding versions of the encodings.

In [7]:
with torch.no_grad():
    encodings = vqvae.encode(video)
    video_recon = vqvae.decode(encodings)
    video_recon = torch.clamp(video_recon, -0.5, 0.5)

## Visualizing Reconstructions

In [8]:
videos = torch.cat((video, video_recon), dim=-1)
videos = videos[0].permute(1, 2, 3, 0) # CTHW -> THWC
videos = ((videos + 0.5) * 255).cpu().numpy().astype('uint8')

fig = plt.figure()
plt.title('real (left), reconstruction (right)')
plt.axis('off')
im = plt.imshow(videos[0, :, :, :])
plt.close()

def init():
    im.set_data(videos[0, :, :, :])

def animate(i):
    im.set_data(videos[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=videos.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())

# Using Pretrained VideoGPT Models

The current available model to download is `ucf101`.

In [9]:
device = torch.device('cuda')
gpt = load_videogpt('ucf101_uncond_gpt', device=device).to(device)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1QkF_Sb2XVRgSbFT_SxQ6aZUeDFoliPQq
From (redirected): https://drive.google.com/uc?id=1QkF_Sb2XVRgSbFT_SxQ6aZUeDFoliPQq&confirm=t&uuid=9b51f87c-4293-44ba-bdb5-0fcd4746ab45
To: /home/ubuntu/.cache/videogpt/ucf101_uncond_gpt
100%|██████████| 1.81G/1.81G [00:13<00:00, 130MB/s] 
/home/ubuntu/miniconda3/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automatically upgraded your loaded checkpoint from v1.3.3 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_ch

`VideoGPT.sample` method returns generated samples of shape BCTHW in the range [0, 1]

In [10]:
samples = gpt.sample(16) # unconditional model does not require batch input

100%|██████████| 4096/4096 [01:44<00:00, 39.34it/s]


In [11]:
import math
import numpy as np

b, c, t, h, w = samples.shape
samples = samples.permute(0, 2, 3, 4, 1)
samples = (samples.cpu().numpy() * 255).astype('uint8')

video = np.zeros((t, (1 + h) * 4 + 1, (1 + w) * 4 + 1, c), dtype='uint8')
for i in range(b):
  r, c = i // 4, i % 4
  start_r, start_c = (1 + h) * r, (1 + w) * c
  video[:, start_r:start_r + h, start_c:start_c + w] = samples[i]

fig = plt.figure()
plt.title('ucf101 unconditional samples')
plt.axis('off')
im = plt.imshow(video[0, :, :, :])
plt.close()

def init():
    im.set_data(video[0, :, :, :])

def animate(i):
    im.set_data(video[i, :, :, :])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0], interval=200) # 200ms = 5 fps
HTML(anim.to_html5_video())